# PyESAPI: Data Mining
*By Michael.Folkerts@varian.com*

In [1]:
import pyesapi
import atexit
app = pyesapi.CustomScriptExecutable.CreateApplication('python_demo')
atexit.register(app.Dispose);

## Extracting Structure Data with Pandas
* Open a patient
* Use pandas dataframe to display basic structure info
* Save to file
* **Extra Credit:** Loop through patients and save to sqlite database, query all PTV volumes

In [2]:
patient = app.OpenPatientById('TestUpperBody')

In [17]:
import pandas as pd

structure_dataframe = pd.DataFrame(
    [(s.Id, s.Name, s.DicomType, s.Volume, s.IsHighResolution) for s in patient.StructureSetsLot(0).Structures],
    columns = ('Id', 'Name', 'Dicom Type', 'Volume', 'Is High Res')
)

print("Structures:\n", structure_dataframe)

Structures:
            Id        Name Dicom Type       Volume  Is High Res
0       Heart       Heart        PTV    66.711074        False
1        BODY        BODY   EXTERNAL  9515.747656        False
2      R Lung      R Lung  AVOIDANCE   939.143211        False
3      L Lung      L Lung  AVOIDANCE   760.367520        False
4        Cord        Cord      ORGAN    39.258909        False
5  Total Lung  Total Lung  AVOIDANCE  1705.624076        False
6    Lg Tumor    Lg Tumor        PTV   825.509991        False


In [23]:
with open('./StructureData.csv','w') as f:
    f.write(structure_dataframe.to_csv())

## Saving DVH to HDF5
* Create HDF5 file
* Create group in H5 file
* Get DVH data for a structure
* loop over patients DVH and save data to HDF5 file

In [24]:
import h5py

C:\Users\Varian\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [25]:
h5file = h5py.File("DVH.h5")

In [26]:
patient_group = h5file.create_group(f'{patient.Id}')